## Для начала посмотрим на наши данные, нужно понять с чем мы работаем

### Подключим все необходимые библиотеки

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, accuracy_score, mean_absolute_error
from collections import Counter
import re
import nltk
#import pymorphy2
from ast import literal_eval
import gensim
import math


### Поля данных

* warehouse_id - идентификатор магазина
* product_id - идентификатор продукта
* date - дата
* quantity - кол-во продаж
* id - уникальный идентификатор строки

 

In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/train.csv")
print(train_data.shape)
train_data.head(16)

(78067, 5)


,warehouse_id,product_id,date,quantity,id
0,0,71165,2020-12-02,1,0
1,0,71165,2020-12-03,3,1
2,0,71165,2020-12-07,2,2
3,0,71165,2020-12-08,2,3
4,0,71165,2020-12-09,3,4
5,0,71165,2020-12-10,1,5
6,0,71165,2020-12-12,6,6
7,0,71165,2020-12-13,4,7
8,0,71165,2020-12-14,4,8
9,0,71165,2020-12-15,9,9


### Посмотрим на количество товаров и на их id

In [4]:
product_id = train_data.sort_values(by=['product_id'])['product_id'].unique()
print(len(product_id))
print(product_id[:5])

3288
[71165 71170 71185 71215 71220]


### Посмотрим на тестовую выборку и сделаем некоторые выводы

In [5]:
test_data = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/test.csv")
print(test_data.shape)
test_data.head(15)

(46032, 4)


,date,product_id,warehouse_id,id
0,2021-04-09,71165,0,0
1,2021-04-09,71165,1,1
2,2021-04-09,71170,0,2
3,2021-04-09,71170,1,3
4,2021-04-09,71185,0,4
5,2021-04-09,71185,1,5
6,2021-04-09,71215,0,6
7,2021-04-09,71215,1,7
8,2021-04-09,71220,0,8
9,2021-04-09,71220,1,9


### Сразу хочется посмотреть на warehouse_id и на product_id, так как при разных product_id будут проблемы


In [6]:
product_test_id = test_data.sort_values(by=['product_id'])['product_id'].unique()
print(len(product_test_id))
print(product_test_id[:5])
print(len(set(product_test_id == product_id)))  # Все значения совпадают, проблем нет

3288
[71165 71170 71185 71215 71220]
1


In [7]:
warehouse_id = test_data['warehouse_id'].unique()
print(len(warehouse_id))  # Всего два магазина

2


### В нашем случае всего два магазина и все значения id товаров совпадают. Проблем с данными нет.

### Посмотрим на simple submission

In [8]:
simple_data = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/sub.csv")
print(simple_data.shape)
simple_data.head()

(46032, 2)


,id,quantity
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


## Разделение данных на train / test

### Итак, суть задачи ясна. Проблем пока не обнаружено, поэтому разделяем нашу train выборку и начинаем реализовывать идеи

In [9]:
'''
# Это плохой способ разделять данные в нашем случае!
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('quantity', axis=1), train_data[['quantity']], test_size=0.1945, random_state=42)
print(X_train.head(3))
print(X_test.head(3))
print(y_train.head(3))
print(y_test.head(3))
'''

"\n# Это плохой способ разделять данные в нашем случае!\nX_train, X_test, y_train, y_test = train_test_split(train_data.drop('quantity', axis=1), train_data[['quantity']], test_size=0.1945, random_state=42)\nprint(X_train.head(3))\nprint(X_test.head(3))\nprint(y_train.head(3))\nprint(y_test.head(3))\n"

### Необходимо понимать, что в данных есть пропуски. Это видно по X_train

In [10]:
train_data = train_data.sort_values(by=['date', 'product_id'])
X_train = train_data.drop('quantity', axis=1)
y_train = train_data['quantity']
X_train

,warehouse_id,product_id,date,id
181,0,71220,2020-11-27,191
30017,1,71220,2020-11-27,32102
30551,1,71455,2020-11-27,32660
30772,1,71575,2020-11-27,32892
30796,1,71580,2020-11-27,32917
...,...,...,...,...
78058,1,98240,2021-04-08,83117
78061,1,98250,2021-04-08,83131
29790,0,98255,2021-04-08,31817
78065,1,98620,2021-04-08,83150


### Таким образом имеем отсортированные по дате и id данные. Идея деления:

*   Находим границу последних n дней
*   Делим на train / test по этой границе



In [11]:
# n = 5  # 2021-04-08 - 5 = 2021-04-03 -- Граница
X_train = train_data.where(train_data['date'] < '2021-04-03').dropna().drop('quantity', axis=1)
y_train = train_data.where(train_data['date'] < '2021-04-03').dropna()['quantity']
X_test = train_data.where(train_data['date'] >= '2021-04-03').dropna().drop('quantity', axis=1)
y_test = train_data.where(train_data['date'] >= '2021-04-03').dropna()['quantity']
print(X_train.head(3))
print(X_test.head(3))
print(y_train.head(3))
print(y_test.head(3))

       warehouse_id  product_id        date       id
181             0.0     71220.0  2020-11-27    191.0
30017           1.0     71220.0  2020-11-27  32102.0
30551           1.0     71455.0  2020-11-27  32660.0
       warehouse_id  product_id        date       id
92              0.0     71165.0  2021-04-03     92.0
29894           1.0     71165.0  2021-04-03  31961.0
179             0.0     71215.0  2021-04-03    186.0
181      1.0
30017    1.0
30551    1.0
Name: quantity, dtype: float64
92       7.0
29894    3.0
179      1.0
Name: quantity, dtype: float64


## Первая идея: дни недели

### Суть идеи заключается в том, чтобы в каждом магазине найти некие зависимости в соотношении проданный товар/день недели, далее на основе этих зависимостей седлать прогноз



---





### Для начала создадим словарь: дата - день недели

In [12]:
data_ = pd.Series(pd.to_datetime(train_data['date'].unique()))

# print(data_, type(data_))
data_week = data_.dt.day_name()
# print(data_week)
date2week = dict(zip(train_data['date'].unique(), data_week))
print(date2week)

{'2020-11-27': 'Friday', '2020-11-28': 'Saturday', '2020-11-29': 'Sunday', '2020-11-30': 'Monday', '2020-12-01': 'Tuesday', '2020-12-02': 'Wednesday', '2020-12-03': 'Thursday', '2020-12-04': 'Friday', '2020-12-05': 'Saturday', '2020-12-06': 'Sunday', '2020-12-07': 'Monday', '2020-12-08': 'Tuesday', '2020-12-09': 'Wednesday', '2020-12-10': 'Thursday', '2020-12-11': 'Friday', '2020-12-12': 'Saturday', '2020-12-13': 'Sunday', '2020-12-14': 'Monday', '2020-12-15': 'Tuesday', '2020-12-16': 'Wednesday', '2020-12-17': 'Thursday', '2020-12-18': 'Friday', '2020-12-19': 'Saturday', '2020-12-20': 'Sunday', '2020-12-21': 'Monday', '2020-12-22': 'Tuesday', '2020-12-23': 'Wednesday', '2020-12-24': 'Thursday', '2020-12-25': 'Friday', '2020-12-26': 'Saturday', '2020-12-27': 'Sunday', '2020-12-28': 'Monday', '2020-12-29': 'Tuesday', '2020-12-30': 'Wednesday', '2020-12-31': 'Thursday', '2021-01-01': 'Friday', '2021-01-02': 'Saturday', '2021-01-03': 'Sunday', '2021-01-04': 'Monday', '2021-01-05': 'Tuesda

### Далее создадим словарь, где key = warehouse_id + product_id, а value = новый словарь с key = день недели и value = массив числа продаж

In [13]:
my_dict = {}
# k = 0
for product_info, product_quantity in tqdm(zip(X_train[['warehouse_id', 'product_id', 'date']].values, y_train)):
    '''
    product_info[0] - warehouse_id - идентификатор магазина
    product_info[1] - product_id - идентификатор продукта
    product_info[2] - date - дата
    product_quantity - quantity - кол-во продаж
    '''
    my_id = str(product_info[0]) + str(product_info[1])
    # print(my_id, product_info[0], product_info[1])
    if my_id in my_dict:
        if date2week[product_info[2]] in my_dict[my_id]:
            my_dict[my_id][date2week[product_info[2]]].append(product_quantity)
        else:
             my_dict[my_id][date2week[product_info[2]]] = [product_quantity]
    else:
        my_dict[my_id] = {}
        my_dict[my_id][date2week[product_info[2]]] = [product_quantity]
# my_dict

73665it [00:00, 201719.27it/s]


### Теперь для тестовой выборки будем смотреть в этот словарь и искать ответ в зависимости от данных словаря

In [14]:
answer_round, answer_floor, answer_ceil = [], [], []
nice_product, bad_product = 0, 0
for product_info in tqdm(X_test[['warehouse_id', 'product_id', 'date']].values):
    my_id_first = str(product_info[0]) + str(product_info[1])
    my_id_second = date2week[product_info[2]]
    try:
        my_array = my_dict[my_id_first][my_id_second]  # Если данных нет в словаре 
        nice_product += 1
    except:
        bad_product += 1
        my_array = []
    if len(my_array) < 1:
        answer_round.append(1)
        answer_floor.append(1)
        answer_ceil.append(1)
    else:  # Попробуем среднее
        answer_round.append(round(sum(my_array) / len(my_array)))
        answer_floor.append(math.floor(sum(my_array) / len(my_array)))
        answer_ceil.append(math.ceil(sum(my_array) / len(my_array)))
print()
print(nice_product, bad_product)
# print(len(answer), len(y_test))
print(mean_absolute_error(y_test, answer_round))
print(mean_absolute_error(y_test, answer_floor))
print(mean_absolute_error(y_test, answer_ceil))

100%|██████████| 4402/4402 [00:00<00:00, 148176.03it/s]


3550 852
0.7716946842344389
0.726715129486597
0.8555202180826896


### Показывает достаточно неплохой результат. Причём округление в большую сторону даёт лучший результат. Его и буду использовать. 

In [15]:
data_ = pd.Series(pd.to_datetime(test_data['date'].unique()))

# print(data_, type(data_))
data_week = data_.dt.day_name()
# print(data_week)
date2week = dict(zip(test_data['date'].unique(), data_week))
print(date2week)

{'2021-04-09': 'Friday', '2021-04-10': 'Saturday', '2021-04-11': 'Sunday', '2021-04-12': 'Monday', '2021-04-13': 'Tuesday', '2021-04-14': 'Wednesday', '2021-04-15': 'Thursday'}


In [16]:
answer_ceil = []
nice_product, bad_product = 0, 0
data_2 = pd.Series(pd.to_datetime(test_data['date'].unique()))
for product_info in tqdm(test_data[['warehouse_id', 'product_id', 'date']].values):
    #print()
    #print(product_info)
    #break
    my_id_first = str(float(product_info[0])) + str(float(product_info[1]))
    my_id_second = date2week[product_info[2]]
    try:
        my_array = my_dict[my_id_first][my_id_second]  # Если данных нет в словаре 
        nice_product += 1
    except:
        bad_product += 1
        my_array = []
    if len(my_array) < 3:  # Если товар в определённый день недели купили пару раз, то логично предположить, что его не купят вовсе
        answer_ceil.append(0)

    else:  
        answer_ceil.append(math.ceil(sum(my_array) / len(my_array)))
print()
print(nice_product, bad_product)
print(len(answer_ceil), len(simple_data))


100%|██████████| 46032/46032 [00:00<00:00, 251641.84it/s]


25975 20057
46032 46032


In [17]:
Result_table = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/sub.csv")
Result_table['quantity'] = pd.Series(answer_ceil)
Result_table[["id", "quantity"]].to_csv("FINAL.csv", index=False)

### Score = 0.46886 
### Мягко говоря - ужасно...


## Выводы после первой идеи:

* Корявая обработка нулей, нужно исправлять
* На основе количества продаж в определённые дни нужно пытаться делать что-то более нетривиальное, чем просто среднее
* Необходимо изменить test / train выборку, ибо нет учёта нулей



### Начнём исправляться с заполнения нашей выборки нулями
Идея такая: если в определённый день информации о товаре нет, то значит его купили 0 раз. Это поле должно храниться в выборке.

In [18]:
product_id = train_data['product_id'].unique()
date = train_data['date'].unique()
warehouse_id = train_data['warehouse_id'].unique()
print(len(product_id), len(date), len(warehouse_id), len(product_id) * len(date) * len(warehouse_id))

3288 133 2 874608


### Значит у нас должно быть 874608 записей. Не будем вставлять недостоющие записи в таблицу, а сразу будем составлять словарик. Он будет таким же, но более разряженее.

In [19]:
my_current_dict = {}
for product_info in tqdm(train_data[['warehouse_id', 'product_id', 'date', 'quantity']].values):
    '''
    product_info[0] - warehouse_id - идентификатор магазина
    product_info[1] - product_id - идентификатор продукта
    product_info[2] - date - дата
    product_info[3] - quantity - кол-во продаж
    '''
    my_id = str(product_info[0]) + str(product_info[1]) + str(product_info[2])
    my_current_dict[my_id] = product_info[3]
print()
print(len(my_current_dict))

100%|██████████| 78067/78067 [00:00<00:00, 430155.21it/s]


78067


### Информация есть менее чем о 10% товаров.

In [20]:
data_ = pd.Series(pd.to_datetime(train_data['date'].unique()))

# print(data_, type(data_))
data_week = data_.dt.day_name()
# print(data_week)
date2week = dict(zip(train_data['date'].unique(), data_week))
print(date2week)

{'2020-11-27': 'Friday', '2020-11-28': 'Saturday', '2020-11-29': 'Sunday', '2020-11-30': 'Monday', '2020-12-01': 'Tuesday', '2020-12-02': 'Wednesday', '2020-12-03': 'Thursday', '2020-12-04': 'Friday', '2020-12-05': 'Saturday', '2020-12-06': 'Sunday', '2020-12-07': 'Monday', '2020-12-08': 'Tuesday', '2020-12-09': 'Wednesday', '2020-12-10': 'Thursday', '2020-12-11': 'Friday', '2020-12-12': 'Saturday', '2020-12-13': 'Sunday', '2020-12-14': 'Monday', '2020-12-15': 'Tuesday', '2020-12-16': 'Wednesday', '2020-12-17': 'Thursday', '2020-12-18': 'Friday', '2020-12-19': 'Saturday', '2020-12-20': 'Sunday', '2020-12-21': 'Monday', '2020-12-22': 'Tuesday', '2020-12-23': 'Wednesday', '2020-12-24': 'Thursday', '2020-12-25': 'Friday', '2020-12-26': 'Saturday', '2020-12-27': 'Sunday', '2020-12-28': 'Monday', '2020-12-29': 'Tuesday', '2020-12-30': 'Wednesday', '2020-12-31': 'Thursday', '2021-01-01': 'Friday', '2021-01-02': 'Saturday', '2021-01-03': 'Sunday', '2021-01-04': 'Monday', '2021-01-05': 'Tuesda

In [21]:
my_huge_dict = {}
nice_product = 0
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            my_id = str(id_warehouse) + str(id_product) + str(id_date)
            if my_id in my_current_dict:
                nice_product += 1
                if id_warehouse in my_huge_dict:
                    if id_product in my_huge_dict[id_warehouse]:
                        if date2week[id_date] in my_huge_dict[id_warehouse][id_product]:
                            my_huge_dict[id_warehouse][id_product][date2week[id_date]].append(my_current_dict[my_id])
                        else:
                            my_huge_dict[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                    else:
                        my_huge_dict[id_warehouse][id_product] = {}
                        my_huge_dict[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                else:
                    my_huge_dict[id_warehouse] = {}
                    my_huge_dict[id_warehouse][id_product] = {}
                    my_huge_dict[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
            else:
                if id_warehouse in my_huge_dict:
                    if id_product in my_huge_dict[id_warehouse]:
                        if date2week[id_date] in my_huge_dict[id_warehouse][id_product]:
                            my_huge_dict[id_warehouse][id_product][date2week[id_date]].append(0)
                        else:
                            my_huge_dict[id_warehouse][id_product][date2week[id_date]] = [0]
                    else:
                        my_huge_dict[id_warehouse][id_product] = {}
                        my_huge_dict[id_warehouse][id_product][date2week[id_date]] = [0]
                else:
                    my_huge_dict[id_warehouse] = {}
                    my_huge_dict[id_warehouse][id_product] = {}
                    my_huge_dict[id_warehouse][id_product][date2week[id_date]] = [0]
print()
print(nice_product)

100%|██████████| 3288/3288 [00:01<00:00, 2281.97it/s]


78067


### Проверим себя

In [22]:
amount = 0
set_amount = set()
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            if str(id_warehouse) + str(id_product) + str(date2week[id_date]) not in set_amount:
                amount += len(my_huge_dict[id_warehouse][id_product][date2week[id_date]])
                set_amount.add(str(id_warehouse) + str(id_product) + str(date2week[id_date]))
print()
print(amount)

100%|██████████| 3288/3288 [00:00<00:00, 3394.96it/s]


874608


### Размеры совпадают, теперь можно делить на train / test. За границу возьмём 2021-04-01, ровно 7 дней

In [23]:
# Понимаю, что проделываю такую же работу, что и пару ячеек выше. Но хочу сделать акцент на понимании кода
my_huge_dict_train = {}
my_huge_dict_test = {}
nice_product = 0
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            if id_date > '2021-04-01':
                my_id = str(id_warehouse) + str(id_product) + str(id_date)
                if my_id in my_current_dict:
                    nice_product += 1
                    if id_warehouse in my_huge_dict_test:
                        if id_product in my_huge_dict_test[id_warehouse]:
                            if date2week[id_date] in my_huge_dict_test[id_warehouse][id_product]:
                                my_huge_dict_test[id_warehouse][id_product][date2week[id_date]].append(my_current_dict[my_id])
                            else:
                                my_huge_dict_test[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                        else:
                            my_huge_dict_test[id_warehouse][id_product] = {}
                            my_huge_dict_test[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                    else:
                        my_huge_dict_test[id_warehouse] = {}
                        my_huge_dict_test[id_warehouse][id_product] = {}
                        my_huge_dict_test[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                else:
                    if id_warehouse in my_huge_dict_test:
                        if id_product in my_huge_dict_test[id_warehouse]:
                            if date2week[id_date] in my_huge_dict_test[id_warehouse][id_product]:
                                my_huge_dict_test[id_warehouse][id_product][date2week[id_date]].append(0)
                            else:
                                my_huge_dict_test[id_warehouse][id_product][date2week[id_date]] = [0]
                        else:
                            my_huge_dict_test[id_warehouse][id_product] = {}
                            my_huge_dict_test[id_warehouse][id_product][date2week[id_date]] = [0]
                    else:
                        my_huge_dict_test[id_warehouse] = {}
                        my_huge_dict_test[id_warehouse][id_product] = {}
                        my_huge_dict_test[id_warehouse][id_product][date2week[id_date]] = [0]
            else:
                my_id = str(id_warehouse) + str(id_product) + str(id_date)
                if my_id in my_current_dict:
                    nice_product += 1
                    if id_warehouse in my_huge_dict_train:
                        if id_product in my_huge_dict_train[id_warehouse]:
                            if date2week[id_date] in my_huge_dict_train[id_warehouse][id_product]:
                                my_huge_dict_train[id_warehouse][id_product][date2week[id_date]].append(my_current_dict[my_id])
                            else:
                                my_huge_dict_train[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                        else:
                            my_huge_dict_train[id_warehouse][id_product] = {}
                            my_huge_dict_train[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                    else:
                        my_huge_dict_train[id_warehouse] = {}
                        my_huge_dict_train[id_warehouse][id_product] = {}
                        my_huge_dict_train[id_warehouse][id_product][date2week[id_date]] = [my_current_dict[my_id]]
                else:
                    if id_warehouse in my_huge_dict_train:
                        if id_product in my_huge_dict_train[id_warehouse]:
                            if date2week[id_date] in my_huge_dict_train[id_warehouse][id_product]:
                                my_huge_dict_train[id_warehouse][id_product][date2week[id_date]].append(0)
                            else:
                                my_huge_dict_train[id_warehouse][id_product][date2week[id_date]] = [0]
                        else:
                            my_huge_dict_train[id_warehouse][id_product] = {}
                            my_huge_dict_train[id_warehouse][id_product][date2week[id_date]] = [0]
                    else:
                        my_huge_dict_train[id_warehouse] = {}
                        my_huge_dict_train[id_warehouse][id_product] = {}
                        my_huge_dict_train[id_warehouse][id_product][date2week[id_date]] = [0]
print()
print(nice_product)

100%|██████████| 3288/3288 [00:01<00:00, 2237.93it/s]


78067


In [24]:
amount = 0
set_amount = set()
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            if str(id_warehouse) + str(id_product) + str(date2week[id_date]) not in set_amount:
                amount += len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])
                set_amount.add(str(id_warehouse) + str(id_product) + str(date2week[id_date]))
print()
print(amount)

100%|██████████| 3288/3288 [00:00<00:00, 3404.00it/s]


828576


In [25]:
amount = 0
set_amount = set()
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            if str(id_warehouse) + str(id_product) + str(date2week[id_date]) not in set_amount:
                amount += 1  # len(my_huge_dict_test[id_warehouse][id_product][date2week[id_date]])  # Работает, так как у нас всегда массив из одного числа
                set_amount.add(str(id_warehouse) + str(id_product) + str(date2week[id_date]))
print()
print(amount)

100%|██████████| 3288/3288 [00:00<00:00, 3356.73it/s]


46032


### Тестовая выборка занимает всего 5%. Это нужно будет учитывать!

In [26]:
right_answer = []
answer_round, answer_floor, answer_ceil = [], [], []
set_amount = set()
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            right_answer.append(my_huge_dict_test[id_warehouse][id_product][date2week[id_date]][0])
            answer_round.append(round(sum(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])))
            answer_floor.append(math.floor(sum(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])))
            answer_ceil.append(math.ceil(sum(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])))
print(mean_absolute_error(right_answer, answer_round))
print(mean_absolute_error(right_answer, answer_floor))
print(mean_absolute_error(right_answer, answer_ceil))

100%|██████████| 3288/3288 [00:02<00:00, 1637.98it/s]


0.21154848800834203
0.1989051094890511
0.5856795272853667


### Не будем повторять ошибок. Вместо ceil возьмём floor

In [39]:
data_ = pd.Series(pd.to_datetime(test_data['date'].unique()))


#print(data_, type(data_))
data_week = data_.dt.day_name()
# print(data_week)
date2week = dict(zip(test_data['date'].unique(), data_week))
print(date2week)

{'2021-04-09': 'Friday', '2021-04-10': 'Saturday', '2021-04-11': 'Sunday', '2021-04-12': 'Monday', '2021-04-13': 'Tuesday', '2021-04-14': 'Wednesday', '2021-04-15': 'Thursday'}


In [46]:
answer = []
for product_info in tqdm(test_data[['warehouse_id', 'product_id', 'date']].values):
    id_warehouse = (product_info[0])
    id_product = (product_info[1])
    id_date = (product_info[2])
    #print(id_warehouse)
    #print(id_product)
    #print(id_date)
    #print(my_huge_dict[id_warehouse][id_product][date2week[id_date]])
    #break
    answer.append(math.floor(sum(my_huge_dict[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict[id_warehouse][id_product][date2week[id_date]])))
    
print()
print(len(answer), len(simple_data))


100%|██████████| 46032/46032 [00:00<00:00, 250264.37it/s]


46032 46032


In [47]:
Result_table = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/sub.csv")
Result_table['quantity'] = pd.Series(answer)
Result_table[["id", "quantity"]].to_csv("FINAL.csv", index=False)

### Score = 0.20210
### Это уже гораздо лучше, но не предел :)

## Вторая идея: модернизация среднего



### Пока оставим идею с днями недели, но будем рассматривать не обычное среднее, а что-то поинтереснее

In [49]:
data_ = pd.Series(pd.to_datetime(train_data['date'].unique()))

# print(data_, type(data_))
data_week = data_.dt.day_name()
# print(data_week)
date2week = dict(zip(train_data['date'].unique(), data_week))
print(date2week)

{'2020-11-27': 'Friday', '2020-11-28': 'Saturday', '2020-11-29': 'Sunday', '2020-11-30': 'Monday', '2020-12-01': 'Tuesday', '2020-12-02': 'Wednesday', '2020-12-03': 'Thursday', '2020-12-04': 'Friday', '2020-12-05': 'Saturday', '2020-12-06': 'Sunday', '2020-12-07': 'Monday', '2020-12-08': 'Tuesday', '2020-12-09': 'Wednesday', '2020-12-10': 'Thursday', '2020-12-11': 'Friday', '2020-12-12': 'Saturday', '2020-12-13': 'Sunday', '2020-12-14': 'Monday', '2020-12-15': 'Tuesday', '2020-12-16': 'Wednesday', '2020-12-17': 'Thursday', '2020-12-18': 'Friday', '2020-12-19': 'Saturday', '2020-12-20': 'Sunday', '2020-12-21': 'Monday', '2020-12-22': 'Tuesday', '2020-12-23': 'Wednesday', '2020-12-24': 'Thursday', '2020-12-25': 'Friday', '2020-12-26': 'Saturday', '2020-12-27': 'Sunday', '2020-12-28': 'Monday', '2020-12-29': 'Tuesday', '2020-12-30': 'Wednesday', '2020-12-31': 'Thursday', '2021-01-01': 'Friday', '2021-01-02': 'Saturday', '2021-01-03': 'Sunday', '2021-01-04': 'Monday', '2021-01-05': 'Tuesda

In [54]:
import statistics
from scipy import stats as s

right_answer = []
answer_round, answer_floor, answer_ceil = [], [], []
answer_median, answer_median_low, answer_median_high = [], [], []
answer_mode = []
set_amount = set()
for id_warehouse in warehouse_id:
    for id_product in tqdm(product_id):
        for id_date in date:
            right_answer.append(my_huge_dict_test[id_warehouse][id_product][date2week[id_date]][0])
            answer_round.append(round(sum(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])))
            answer_floor.append(math.floor(sum(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])))
            answer_ceil.append(math.ceil(sum(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])/len(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])))

            answer_median.append(statistics.median(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]]))
            answer_median_low.append(statistics.median_low(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]]))
            answer_median_high.append(statistics.median_high(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]]))
            
            answer_mode.append(int(s.mode(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]])[0]))

print(mean_absolute_error(right_answer, answer_round))
print(mean_absolute_error(right_answer, answer_floor))
print(mean_absolute_error(right_answer, answer_ceil))

print(mean_absolute_error(right_answer, answer_median))
print(mean_absolute_error(right_answer, answer_median_low))
print(mean_absolute_error(right_answer, answer_median_high))

print(mean_absolute_error(right_answer, answer_mode))

100%|██████████| 3288/3288 [01:04<00:00, 50.91it/s]


0.21154848800834203
0.1989051094890511
0.5856795272853667
0.19765597844977406
0.19797097671185263
0.1975147723322906
0.20550921098366354


### Медиана даёт неплохой результат. Но давайте попробуем нечто сложнее


In [56]:
def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result[-1]

In [67]:

cur_min = 1
cur_alpha = 0

for i in tqdm(range(1, 30)):
    alpha = i/100
    right_answer = []
    answer_exponential_smoothing_round, answer_exponential_smoothing_floor, answer_exponential_smoothing_ceil = [], [], []
    for id_warehouse in warehouse_id:
        for id_product in (product_id):
            for id_date in date:
                right_answer.append(my_huge_dict_test[id_warehouse][id_product][date2week[id_date]][0])
                #answer_exponential_smoothing_round.append(round(exponential_smoothing(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]], alpha)))
                answer_exponential_smoothing_floor.append(math.floor(exponential_smoothing(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]], alpha)))
                #answer_exponential_smoothing_ceil.append(math.ceil(exponential_smoothing(my_huge_dict_train[id_warehouse][id_product][date2week[id_date]], alpha)))
    if mean_absolute_error(right_answer, answer_exponential_smoothing_floor) < cur_min:
        cur_min = mean_absolute_error(right_answer, answer_exponential_smoothing_floor)
        cur_alpha = alpha
#print(mean_absolute_error(right_answer, answer_exponential_smoothing_round))
print(cur_min, cur_alpha)
#print(mean_absolute_error(right_answer, answer_exponential_smoothing_ceil))


100%|██████████| 29/29 [02:38<00:00,  5.45s/it]

0.19703684393465415 0.14


### Сильного прироста нет, поэтому двойное экспонициальное сглаживание использовать не имеет смысла

In [68]:
data_ = pd.Series(pd.to_datetime(test_data['date'].unique()))


#print(data_, type(data_))
data_week = data_.dt.day_name()
# print(data_week)
date2week = dict(zip(test_data['date'].unique(), data_week))
print(date2week)

{'2021-04-09': 'Friday', '2021-04-10': 'Saturday', '2021-04-11': 'Sunday', '2021-04-12': 'Monday', '2021-04-13': 'Tuesday', '2021-04-14': 'Wednesday', '2021-04-15': 'Thursday'}


In [69]:
answer = []
alpha = 0.14
for product_info in tqdm(test_data[['warehouse_id', 'product_id', 'date']].values):
    id_warehouse = (product_info[0])
    id_product = (product_info[1])
    id_date = (product_info[2])
    answer.append(math.floor(exponential_smoothing(my_huge_dict[id_warehouse][id_product][date2week[id_date]], alpha)))
    
print()
print(len(answer), len(simple_data))


100%|██████████| 46032/46032 [00:00<00:00, 109963.03it/s]


46032 46032


In [71]:
Result_table = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/sub.csv")
Result_table['quantity'] = pd.Series(answer)
Result_table[["id", "quantity"]].to_csv("FINAL_exp.csv", index=False)

### Score = 0.20085
### Сильного роста нет. Нужно что-то сложнее...